In [15]:
# Import Modules
import pandas as pd
import random
from datetime import datetime,timedelta


In [16]:
fake = Faker()
# Increase the size of the customer base

Banks=['Axis Bank','HDFC Bank','ICICI Bank','SBI Bank','PNB Bank','Kotak Mahindra Bank']
CardTypes=['Visa','Rupay','MasterCard']
AtmID=['ATM001','ATM002','ATM003','ATM004','ATM005','ATM006','ATM007','ATM008','ATM009','ATM010']


In [5]:
# Function to genrate Random Transactions for 100,000 users...
def LargeDataset(NumRecords):
    dataset=[]
    startdate=datetime(2024,10,30,0,0,0)
    global CustomerName
    for i in range(NumRecords):
        CardNum=random.randint(10000,99999)
        random_name=random.choice(CustomerName)
        CustomerName=f"{random_name}{i % len(CustomerName)}"
        TransactionDateTime=startdate + timedelta(minutes=random.randint(1,1440))
        TransactionAmount=round(random.uniform(500,50000),2)
        Status=random.choice(['S','F'])  # 'S' for Success  & 'F' for Failure
        CardType=random.choice(CardTypes)
        BankName=random.choice(Banks)
        AtmId=random.choice(AtmID)

        # Now we have to append this data into the dataset which we have created above inside function

        dataset.append({
            "CardNum":CardNum,
            "CustomerName":CustomerName,
            "TransactionDate&Time":TransactionDateTime,
            "TransactionAmount":TransactionAmount,
            "Status":Status,
            "CardType":CardType,
            "BankName":BankName,
            "AtmId":AtmId
        })

    # Now we have to return this datset in form of dataframe using pandas library
    return pd.DataFrame(dataset)

# Generate Dataset with 100,000 transactions(10 transaction per user on average)
dfLarge=LargeDataset(100000)
dfLarge.head()
dfLarge.to_excel("Sam.xlsx")
print("Save Sucess")

Save Sucess


## create 2 extra columns

In [6]:
# Define The Thresholds
HighValueThreshold=10000  # Example Threshols for High Values Transactions

def IdentifyFraudLogic(df):
    #Suspected fraud transaction : High-value or failed transactions
    df['SuspectedFraud']=df.apply(lambda row: True if row['TransactionAmount']>HighValueThreshold or row['Status']=='F' else False,axis=1)

    # Cyclic transactions: Multiple Transaction by the same user with the same amount in a short period
    CyclicTransaction=[]
    for Name,Group in df.groupby('CustomerName'):
        Amounts=Group["TransactionAmount"].value_counts()
        CyclicTransaction+=Group[Group['TransactionAmount'].isin(Amounts[Amounts>1].index)].index.tolist()

    df['CyclicTransactions']=df.index.isin(CyclicTransaction)
    
    return df

dfFraudLarge=IdentifyFraudLogic(dfLarge)
dfFraudLarge[dfFraudLarge['SuspectedFraud'] | dfFraudLarge['CyclicTransactions']].head()

,CardNum,CustomerName,TransactionDate&Time,TransactionAmount,Status,CardType,BankName,AtmId,SuspectedFraud,CyclicTransactions
1,19211,>1,2024-10-30 09:24:00,46195.67,F,Visa,PNB Bank,ATM003,True,False
2,87385,>0,2024-10-30 06:43:00,24722.28,S,Visa,SBI Bank,ATM005,True,False
3,44452,>1,2024-10-30 17:46:00,21971.84,F,Visa,Kotak Mahindra Bank,ATM010,True,False
4,24231,>0,2024-10-30 15:35:00,35704.96,F,Visa,Axis Bank,ATM007,True,False
5,76493,>1,2024-10-30 09:19:00,11063.51,F,MasterCard,SBI Bank,ATM007,True,False


## data encoding

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder as LE

# Encoding categorical variables
dfMLLarge=dfFraudLarge.copy()
LabelEncoder=LE() # Giving a short name to Label Encoder to LE

dfMLLarge['CustomerName']=LabelEncoder.fit_transform(dfMLLarge['CustomerName'])
dfMLLarge['CardType']=LabelEncoder.fit_transform(dfMLLarge['CardType'])
dfMLLarge['BankName']=LabelEncoder.fit_transform(dfMLLarge['BankName'])
dfMLLarge['AtmId']=LabelEncoder.fit_transform(dfMLLarge['AtmId'])

# Convert TransactionDate&Time to TimeStamp
dfMLLarge['TransactionDate&Time']=pd.to_datetime(dfFraudLarge['TransactionDate&Time']).astype(int)//10**9


# train test split

In [8]:
# model data test train split
# Define Features and target..
x=dfMLLarge.drop(columns=['Status'])
y=dfMLLarge['Status']
y= y.map({'F':0,'S':1})
# Split the dataset into training and test series
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

# Check the size of the training and test datasets...
x_train.shape,x_test.shape

((80000, 9), (20000, 9))

In [9]:
y_train

75220    1
48955    0
44966    1
13568    1
92727    1
        ..
6265     1
54886    0
76820    1
860      1
15795    1
Name: Status, Length: 80000, dtype: int64

# model Creations

In [10]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score



# Train the RandomForestClassifier
Model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=1)
Model.fit(x_train, y_train)

# Predict on the test set
y_pred = Model.predict(x_test)

# Evaluate the Model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.6034
              precision    recall  f1-score   support

           0       0.60      0.62      0.61      9994
           1       0.61      0.59      0.60     10006

    accuracy                           0.60     20000
   macro avg       0.60      0.60      0.60     20000
weighted avg       0.60      0.60      0.60     20000



In [11]:
y_pred

array([1, 0, 0, ..., 1, 1, 0])

In [12]:
Model.predict

<bound method ForestClassifier.predict of RandomForestClassifier(n_jobs=1, random_state=42)>

## Again create data for predictions

In [13]:
x_test.sample(10)

,CardNum,CustomerName,TransactionDate&Time,TransactionAmount,CardType,BankName,AtmId,SuspectedFraud,CyclicTransactions
75741,78096,1,1730249520,15118.15,2,5,4,True,False
69663,11334,1,1730300340,31681.16,0,2,2,True,False
76236,12283,2,1730288400,23821.17,1,0,5,True,False
53299,14301,3,1730271300,25748.45,2,4,1,True,False
11554,31198,2,1730286660,9274.76,2,5,0,True,False
12743,88495,3,1730316120,9008.46,0,5,9,True,False
69354,97028,0,1730272200,42637.32,1,5,1,True,False
83003,55819,3,1730275860,10117.14,2,1,6,True,False
83121,50126,1,1730268720,34953.32,0,3,5,True,False
3826,50802,2,1730259120,13059.94,0,2,8,True,False


## final predtionctions

In [14]:
print(Model.predict(x_test.sample(10)))

[1 1 0 1 1 1 1 0 1 1]
